In [1]:
import os
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import scipy.cluster.hierarchy as spc

pio.templates.default = "plotly_white"

In [2]:
try:
    _ = first_run
except NameError:
    first_run = True
    os.chdir(os.getcwd().rsplit("/", 1)[0])
    from _aux import functions as func

# Load data

In [3]:
default = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
        usecols=[
            "row_id",
            "account_amount_added_12_24m",
            "sum_capital_paid_account_0_12m",
            "sum_capital_paid_account_12_24m",
            "account_incoming_debt_vs_paid_0_24m",
            "num_unpaid_bills",
            "sum_paid_inv_0_12m",
            "max_paid_inv_0_12m",
            "max_paid_inv_0_24m",
            "num_active_inv",
            "num_active_div_by_paid_inv_0_12m",
        ],
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
    .query("default == 1")
)

not_default = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
        usecols=[
            "row_id",
            "account_amount_added_12_24m",
            "sum_capital_paid_account_0_12m",
            "sum_capital_paid_account_12_24m",
            "account_incoming_debt_vs_paid_0_24m",
            "num_unpaid_bills",
            "sum_paid_inv_0_12m",
            "max_paid_inv_0_12m",
            "max_paid_inv_0_24m",
            "num_active_inv",
            "num_active_div_by_paid_inv_0_12m",
        ],
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
    .query("default == 0")
)

df = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
)

## Correlation between "account" variables

As shown in our sanity profile report, account variables have high correlation amongst themselves mainly due to overlaping lookback windows for aggregation. The first thing we must do is choose one (or some) of them to represent the group, which can be achieved by correlation clustering.

In [4]:
corr = pd.concat([default, not_default]).corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        z=corr.mask(mask),
        x=corr.columns,
        y=corr.columns,
        colorscale=px.colors.diverging.RdBu,
        zmin=-1,
        zmax=1,
    )
)

fig.update_layout(
    title="Correlation between 'account' variables",
    yaxis_autorange="reversed",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    width=1000,
    height=500,
)

# fig.update_traces(opacity=0.6)
fig.show()

In [5]:
pdist = spc.distance.pdist(corr)
linkage = spc.linkage(pdist, method="single")
idx = spc.fcluster(linkage, 0.5 * pdist.max(), "distance")

columns = [default.columns.tolist()[i] for i in list((np.argsort(idx)))]
clusterd_corr = pd.concat([default, not_default]).reindex(columns, axis=1).corr()

mask = np.zeros_like(clusterd_corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

corr["default"].to_frame(name="corr_with_label").assign(cluster=idx).drop(
    "default"
).sort_values(["cluster", "corr_with_label"], ascending=[True, False])

,corr_with_label,cluster
account_amount_added_12_24m,0.006217,1
sum_capital_paid_account_12_24m,0.005424,1
sum_capital_paid_account_0_12m,0.002648,1
num_unpaid_bills,0.028196,2
num_active_inv,0.018780,2
sum_paid_inv_0_12m,-0.034797,2
max_paid_inv_0_12m,-0.041735,2
max_paid_inv_0_24m,-0.042666,2
num_active_div_by_paid_inv_0_12m,0.137125,3
account_incoming_debt_vs_paid_0_24m,-0.000125,5


Despite our knowing that correlation is not equal to causation, lack of proper time for exploration leads us to the not so ideal path of using correlation to exclude variables. At this time. we choose to analyze only the variable "num_active_div_by_paid_inv_0_12m" at the cost of probably losing some good insights.

## 1. "num_active_div_by_paid_inv_0_12m"

This variable has 22.8% of missing values and 48.7% of zero values, on top of some hard-hitting outliers.
One behaviour this variable describes is whether the customer has had a good experience with us and has come back for more. On the flip side, users that have not finished paying their first invoice yet (in a 12 month span) are attributed NaN simple because division by zero is undefined. Hence, a good customer will get NaN on their first purchase, but also an ill-intentioned customer that executes multiple orders with no intention to pay them. Therefore, we choose to fill missing values with the number of active invoices so that the latter can become visible through this variable.

Next, we look at how defaults are distributed across the categories.


In [20]:
df[["num_active_div_by_paid_inv_0_12m"]].assign(
    num_active_div_by_paid_inv_0_12m=df["num_active_div_by_paid_inv_0_12m"].combine_first(df["num_active_inv"])
).describe(np.append(np.arange(.25, 1., .1), np.array([.99])))

,num_active_div_by_paid_inv_0_12m
count,71980.000000
mean,0.112705
std,0.310636
min,0.000000
25%,0.000000
35%,0.000000
45.0%,0.000000
50%,0.000000
55.0%,0.000000
65.0%,0.000000


In [44]:
_, var_profile = func.test_k_prop(
    df[["num_active_div_by_paid_inv_0_12m", "default"]].assign(
        num_active_div_by_paid_inv_0_12m=df["num_active_div_by_paid_inv_0_12m"].combine_first(df["num_active_inv"]),
        bins_var=pd.cut(df["num_active_div_by_paid_inv_0_12m"], range(-1, 10))
    )
    .groupby("bins_var")["default"]
    .agg(["sum", "count"])
    .assign(
        default=lambda df: df["sum"], not_default=lambda df: df["count"] - df["sum"]
    )[["default", "not_default", "count"]]
    .transform(lambda s: s.astype(int))
)

var_profile

Using 9 degrees of freedom
Reject null hypothesis with 1129.173122989313 > 16.91897760462045


,default,not_default,count,expected_default,expected_not_default,chi_default,chi_not_default
bins_var,,,,,,,
"(-1, 0]",283,34788,35071,367.493464,34703.506536,19.426592,0.205718
"(0, 1]",245,19749,19994,209.508264,19784.491736,6.012476,0.063669
"(1, 2]",29,362,391,4.097116,386.902884,151.363469,1.602866
"(2, 3]",12,47,59,0.618235,58.381765,209.539430,2.218922
"(3, 4]",3,8,11,0.115264,10.884736,72.196801,0.764529
"(4, 5]",5,4,9,0.094307,8.905693,255.185945,2.702297
"(5, 6]",1,2,3,0.031436,2.968564,29.842432,0.316017
"(6, 7]",1,0,1,0.010479,0.989521,93.443468,0.989521
"(7, 8]",1,0,1,0.010479,0.989521,93.443468,0.989521


In [55]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=var_profile.reset_index().index.to_list(),
        y=(var_profile["not_default"] / var_profile["count"]),
        name="not_default",
    )
)

fig.add_trace(
    go.Bar(
        x=var_profile.reset_index().index.to_list(),
        y=(var_profile["default"] / var_profile["count"]),
        name="default",
    )
)

fig.update_layout(
    title="Distribution of observations across 'num_active_div_by_paid_inv_0_12m' classes",
    barmode="relative",
    yaxis_title="Percentage",
    yaxis_tickformat="%",
    xaxis_title="Ordered Bins",
    xaxis_tickvals=[x for x in range(0, var_profile.shape[0])]
)

fig.update_traces(opacity=0.75)
fig.show()